In [1]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
# train : train set을 불러올지 test set을 불러올지
# transform : 어떤 transform은 어떤 것을 적용할지
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
# dataset : 어떤 데이터를 불러올지
# drop_last : 마지막 batch를 버릴지 말지
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [9]:
# MNIST data image of shape 28 * 28 = 784
# 입력 개수, 출력개수(0-9까지의 class가짐) , bias 
linear = torch.nn.Linear(784, 10, bias=True).to(device)

In [10]:
# Initialization 
# weight를 normal distribution으로 초기화
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[ 1.0600, -1.1635,  0.6505,  ..., -0.4968,  0.9855, -0.2384],
        [-0.4499, -2.1725, -0.1246,  ..., -0.3793, -0.4681, -0.1758],
        [ 0.4246,  1.5228,  0.3058,  ..., -1.8634,  0.2990, -1.3663],
        ...,
        [ 2.0748,  0.0854, -0.3922,  ..., -0.0590, -0.3197, -1.3612],
        [-0.5007, -0.9762,  0.3155,  ...,  0.9442,  1.2646, -0.6966],
        [-0.9794, -1.1702, -0.8225,  ..., -0.3833, -0.6096,  0.6062]],
       requires_grad=True)

In [11]:
# define cost/loss & optimizer
# cross entropy loss 정의
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.

# 이부분이 다름, SGD를 사용했었는데 다양한 optimization 중 Adam을 사용함
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

In [12]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    # X : data
    # Y : label
    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

		# gradient 구하기
        optimizer.zero_grad()
        # 예측값
        hypothesis = linear(X)
        # 예측값과 실제값의 cost 구하기
        cost = criterion(hypothesis, Y)
        # backpropogation
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 4.797653198
Epoch: 0002 cost = 1.671702623
Epoch: 0003 cost = 1.111340404
Epoch: 0004 cost = 0.875707567
Epoch: 0005 cost = 0.744093716
Epoch: 0006 cost = 0.659061313
Epoch: 0007 cost = 0.598850131
Epoch: 0008 cost = 0.553448677
Epoch: 0009 cost = 0.519176483
Epoch: 0010 cost = 0.490894258
Epoch: 0011 cost = 0.468347549
Epoch: 0012 cost = 0.448918343
Epoch: 0013 cost = 0.432120293
Epoch: 0014 cost = 0.417880177
Epoch: 0015 cost = 0.405863255
Learning finished


In [13]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8859000205993652
Label:  8
Prediction:  3


/opt/conda/lib/python3.6/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/opt/conda/lib/python3.6/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
